In [1]:
import torch
from torch import nn
from pathlib import Path
import numpy as np
import seaborn as sns
import scipy
import matplotlib.pyplot as plt
plt.style.use('dark_background')

from modules import ELBO
from utils import load_sim_data

### Setup parameters

In [2]:
N = 11
out_dict = "/Users/nguyentiendung/GitHub/perceptual-straightening/model/"
data_path = "/Users/nguyentiendung/GitHub/perceptual-straightening/data/simulations/"

### Run inference

In [3]:
elbo = ELBO(N, data_path)

lr = 1e-4
optimizer = torch.optim.Adam(elbo.parameters(), lr=lr)
model_file = Path(out_dict) / 'model.pt'

In [ ]:
iterations = 1000
errors = np.zeros(iterations)
kl_loss = np.zeros(iterations)
ll_loss = np.zeros(iterations)

for i in range(iterations):
    log_ll, d, c, a, l = elbo.compute_likelihood()
    # kl = elbo.kl_divergence()
    # loss = elbo.compute_loss(log_ll, kl)

    # loss.backward()
    log_ll = -log_ll
    log_ll.backward()
    optimizer.step()
    optimizer.zero_grad()

    # errors[i] = loss.item()
    # kl_loss[i] = kl.item()
    ll_loss[i] = log_ll.item()

    # if not i % 100:
    #     print(f"Epoch: {i}, Loss: {loss.item()}")

In [ ]:
fig, ax1 = plt.subplots()

# Primary y-axis (ELBO and Neg. log likelihood)
ax1.plot(errors, label='ELBO')
ax1.plot(ll_loss, label='Neg. log likelihood')
ax1.set_xlabel('Iteration')
ax1.set_ylabel('ELBO / Neg. log likelihood')
ax1.tick_params(axis='y')

# Add title
plt.title('Loss over iterations')

# Create a secondary y-axis for KL loss
ax2 = ax1.twinx()
ax2.plot(kl_loss, label='KL', color='orange')
ax2.set_ylabel('KL Loss')
ax2.tick_params(axis='y')

# Add legends for both axes
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Show the plot
plt.show()

In [ ]:
# # load simulated data
# discrim_mat = torch.from_numpy(scipy.io.loadmat(Path(data_path) / 'discriminality_matrix.mat')['Pc_reshaped'])

# # plot data
# ax = sns.heatmap(discrim_mat, cmap='Blues')
# ax.invert_yaxis()
# plt.show()